# Importing Libraries

In [15]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2
import numpy as np

In [ ]:
path_to_dataset = os.path.join(os.getcwd(), "dataset", "animals")
for dirname, _, filenames in os.walk(path_to_dataset):
    print(dirname)

In [22]:
folder_path = os.path.join(os.getcwd(), "dataset", "animals")
df = pd.DataFrame(columns=["path","label"])
x, y = [], []
target_size = (108, 108, 3)
for subfolder in os.listdir(folder_path):
    for file_name in os.listdir(os.path.join(folder_path, subfolder)):
        file_path = f'{folder_path}/{subfolder}/{file_name}'
        image = load_img(file_path, target_size=target_size)
        img_arr = img_to_array(image)
        df = df.append({"path":img_arr,"label":subfolder},ignore_index=True)



In [23]:
df

,path,label
0,"[[[82.0, 133.0, 164.0], [81.0, 134.0, 165.0], ...",cats
1,"[[[128.0, 127.0, 132.0], [59.0, 77.0, 113.0], ...",cats
2,"[[[178.0, 176.0, 177.0], [148.0, 146.0, 147.0]...",cats
3,"[[[48.0, 38.0, 37.0], [50.0, 42.0, 40.0], [46....",cats
4,"[[[59.0, 74.0, 45.0], [47.0, 62.0, 33.0], [44....",cats
...,...,...
2995,"[[[135.0, 139.0, 102.0], [75.0, 83.0, 32.0], [...",panda
2996,"[[[44.0, 76.0, 9.0], [41.0, 54.0, 26.0], [31.0...",panda
2997,"[[[69.0, 100.0, 92.0], [74.0, 113.0, 130.0], [...",panda
2998,"[[[23.0, 17.0, 19.0], [159.0, 130.0, 114.0], [...",panda
